In [1]:
from ezc3d import c3d
from pyomeca import Markers, Analogs
import config as cfg
import importlib
import numpy as np

def normalize_emg(emg):
    """
    Funkcja do normalizacji sygnalu emg
    """
    emg_p = (
    emg.meca.band_pass(order=2, cutoff=[10, 425])
    .meca.center()
    .meca.abs()
    .meca.low_pass(order=4, cutoff=5, freq=emg.rate)
    .meca.normalize(ref=None, scale=1)
    .meca.time_normalize(n_frames=425)
    )
    return emg_p

def name_body(data_path, marker_list):
    """
    Funkcja dodajaca numer badanego do nazwy markerow (potrzebne do wczytywania konkretnych markerów z pliku)
    """
    s = data_path.split('\\')[-1]
    body = s.split('-')[3]+":"
    for i in range(len(marker_list)):
        marker_list[i] = body+marker_list[i]
    return marker_list

def data_markers(data_path, marker_list):
    """
    Funkcja wczytujaca markery z pliku i zwracajaca tablice z danymi
    """
    frame_rate = 425  #Liczba klatek uzyta do normalizacji 
    
    #Wczytanie, normalizacja
    for i in range(len(marker_list)):
        tmp_markers = Markers.from_c3d(data_path, usecols=[marker_list[i]])
        tmp_markers = tmp_markers.meca.time_normalize(n_frames=frame_rate)
        tmp_markers = tmp_markers.meca.to_wide_dataframe()
        data_markers = data_markers.join(tmp_markers)      
        
    cols = [c for c in data_markers.columns if c.lower()[:4] != 'ones']
    markers_dataframe = data_markers[cols]
    return markers_dataframe

def dataframe(data_path):
    mark_channels = plat_channels = ang_channels = forc_channels = mome_channels = []
    
    mark_channels = name_body(data_path,cfg.marker_channels[:13])
    plat_channels = name_body(data_path,cfg.platform_channels[:])
    ang_channels = name_body(data_path,cfg.angles_channels[:])
    forc_channels = name_body(data_path,cfg.forces_channels[:])
    mome_channels = name_body(data_path,cfg.moments_channels[:])
    powe_channels = name_body(data_path,cfg.powers_channels[:])
 
    try:    
        full_data = data_markers(data_path,mark_channels)
        full_data = full_data.join(data_markers(data_path,cfg.marker_channels[13:]))
        emg1 = Analogs.from_c3d(data_path, usecols=cfg.emg_channels[:9])
        emg2 = Analogs.from_c3d(data_path, usecols=cfg.emg_channels[9:])
        norm_emg1 = normalize_emg(emg1)
        norm_emg2 = normalize_emg(emg2)
        norm_emg1 = norm_emg1.meca.to_wide_dataframe()
        norm_emg2 = norm_emg2.meca.to_wide_dataframe()

        full_data = full_data.join(norm_emg1)
        full_data = full_data.join(norm_emg2)

        full_data = full_data.join(data_markers(data_path,plat_channels))
        full_data = full_data.join(data_markers(data_path,ang_channels))
        full_data = full_data.join(data_markers(data_path,forc_channels))    
        full_data = full_data.join(data_markers(data_path,mome_channels))

        data_powers_tmp = data_markers(data_path,powe_channels)
        cols = [c for c in data_powers_tmp.columns if c.lower()[:1] == 'z']
        data_powers = data_powers_tmp[cols]

        full_data = full_data.join(data_powers)
        return full_data
    except:
        print("Blad wystapil w pliku: ",data_path)
       

# Wektor X

In [2]:
importlib.reload(cfg)
data_path = r"Z:\baza\2020-08-05-B0444-S01-E01-T01-K01.c3d"
n = dataframe(data_path)

In [3]:
n.to_csv('out.csv',index=False)

In [12]:
n

,x_B0444:LSHO,y_B0444:LSHO,z_B0444:LSHO,x_B0444:LELB,y_B0444:LELB,z_B0444:LELB,x_B0444:LWRA,y_B0444:LWRA,z_B0444:LWRA,x_B0444:RSHO,...,z_B0444:LAnklePower,z_B0444:RHipPower,z_B0444:RKneePower,z_B0444:RAnklePower,z_B0444:LShoulderPower,z_B0444:LElbowPower,z_B0444:LWristPower,z_B0444:RShoulderPower,z_B0444:RElbowPower,z_B0444:RWristPower
time,,,,,,,,,,,,,,,,,,,,,
0.000000,-551.204773,-470.153076,1578.444458,-875.472168,-492.201996,1496.573242,-1109.193848,-384.259308,1478.422485,-266.426086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.007712,-551.118079,-470.165489,1578.461017,-875.412350,-492.281144,1496.668241,-1109.180020,-384.488877,1478.536656,-266.379359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.015425,-551.029999,-470.177966,1578.476924,-875.349680,-492.361882,1496.761395,-1109.160121,-384.700446,1478.660749,-266.327735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.023137,-550.940406,-470.190544,1578.492134,-875.283634,-492.444463,1496.852555,-1109.133547,-384.891230,1478.796096,-266.270425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.030849,-550.849514,-470.203345,1578.506804,-875.214709,-492.528529,1496.941747,-1109.101694,-385.064352,1478.940676,-266.208066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.239151,-464.487614,-250.981275,1458.414539,-413.538272,-63.269328,1180.553974,-231.133087,-125.470061,1328.027259,-219.953703,...,-0.012048,-0.080834,0.218356,0.211770,0.003790,-0.002225,0.000440,0.002346,0.000083,0.000075
3.246863,-463.886274,-251.431338,1458.650799,-413.430103,-63.505709,1180.917602,-230.647848,-125.586379,1327.986648,-219.143803,...,-0.010734,-0.071864,0.213580,0.188548,0.003117,-0.002439,0.000525,0.005242,0.000008,0.000033
3.254575,-463.304598,-251.868212,1458.863518,-413.327989,-63.733476,1181.256082,-230.162434,-125.684119,1327.931802,-218.373952,...,-0.009413,-0.061329,0.212258,0.155022,0.002302,-0.002540,0.000584,0.007710,-0.000072,-0.000009


In [37]:
c = c3d(data_path)
c['header']['points']

{'size': 227, 'frame_rate': 200.0, 'first_frame': 0, 'last_frame': 656}

In [3]:
import glob
from pathlib import Path


In [ ]:
i = 0
y = np.zeros((3789,425,190))
for path in Path(r'Z:\baza').rglob('*.c3d'):
#     print("Sciezka: ",path, i)
    n = dataframe(str(path))
    y[i] = n
    i+=1

Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E02-T04-K06.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E03-T02-K01.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E03-T02-K05.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E04-T05-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K01.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K02.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K03.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K05.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K06.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K08.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K09.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-05-B0444-S01-E05-T01-K10.c3d
Blad wystapil w pliku:  Z:\baza\20

Blad wystapil w pliku:  Z:\baza\2020-08-06-B0445-S01-E05-T05-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0445-S01-E05-T05-K05.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0445-S01-E05-T05-K06.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0445-S01-E05-T05-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0445-S01-E05-T05-K08.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0445-S01-E05-T05-K09.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0445-S01-E05-T05-K10.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0446-S01-E01-T03-K03.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0446-S01-E01-T05-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0446-S01-E01-T05-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0446-S01-E02-T02-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0446-S01-E02-T03-K06.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0446-S01-E02-T03-K08.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-06-B0446-S01-E02-T03-K10.c3d
Blad wystapil w pliku:  Z:\baza\20

Blad wystapil w pliku:  Z:\baza\2020-08-07-B0447-S01-E05-T03-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0447-S01-E05-T03-K05.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0447-S01-E05-T03-K06.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0447-S01-E05-T06-K02.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0447-S01-E05-T06-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E01-T01-K02.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E01-T01-K03.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E04-T01-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E04-T01-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E04-T02-K01.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E04-T02-K02.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E04-T02-K03.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E04-T02-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0448-S01-E04-T02-K05.c3d
Blad wystapil w pliku:  Z:\baza\20

Blad wystapil w pliku:  Z:\baza\2020-08-07-B0449-S01-E05-T05-K05.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0449-S01-E05-T05-K06.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0449-S01-E05-T05-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0449-S01-E05-T05-K08.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0449-S01-E05-T05-K09.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-07-B0449-S01-E05-T05-K10.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E03-T01-K01.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E04-T01-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E05-T01-K01.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E05-T01-K02.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E05-T01-K03.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E05-T01-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E05-T01-K05.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-10-B0450-S01-E05-T01-K06.c3d
Blad wystapil w pliku:  Z:\baza\20

Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K01.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K02.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K03.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K04.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K05.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K06.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K07.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K08.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K09.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T02-K10.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T03-K01.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T03-K02.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T03-K03.c3d
Blad wystapil w pliku:  Z:\baza\2020-08-12-B0452-S01-E05-T03-K04.c3d
Blad wystapil w pliku:  Z:\baza\20

In [ ]:
y.shape

# W roznych plikach kolumna czasu jest o 1 wieksza niz kolumna danych

In [ ]:
print("Sciezka: ",path)

In [ ]:
c = c3d(data_path)


In [ ]:
c['data']['points'][:]

# Wektor Y

In [7]:
import glob
from pathlib import Path
import config as cfg
import pandas as pd

In [8]:
importlib.reload(cfg)

s = data_path.split('\\')[-1]
body = s.split('-')[3]

i = 0
b = []
m = []
for path in Path(r'Z:\baza').rglob('*.c3d'):
    s = str(path).split('\\')[-1]
    body = s.split('-')[3]
    b.append(cfg.body.index(body))
    m.append((cfg.move).index(s.split('-')[5]))
    
    

In [9]:
d = {'Osoba' : pd.Series(b),
     'Ruch' : pd.Series(m)
        }

In [10]:
df = pd.DataFrame(d)

In [11]:
df

,Osoba,Ruch
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
3781,14,4
3782,14,4
3783,14,4
3784,14,4


In [12]:
x=pd.read_excel('adnotacja.xls', usecols='A:Y',skiprows=1)

In [13]:
x

,Osoba,Ruch,wysokosc jelca,pozycja sztychu,ustawienie stóp,lokcie blisko siebie,ruch noga do poz. koncowej,wyprostoawnie w lokciach,wyrzut rak do przodu,uderzenie odpowiednim ostrzem,Ocena 1,Ocena 2,Ocena 3
0,0,0,1,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,1,1,1,1,1
2,0,2,1,1,1,1,1,1,1,1,1,1,1
3,0,3,1,1,1,1,1,1,1,1,1,1,1
4,0,4,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,14,0,0,1,1,0,1,1,1,1,0,1,1
71,14,1,0,1,1,1,1,1,1,0,0,1,1
72,14,2,1,1,1,0,1,1,1,0,1,1,0
73,14,3,0,1,1,0,0,1,1,0,1,1,1


In [14]:
result = pd.merge(df,x,how='inner',on =['Osoba','Ruch'])

In [15]:
result

,Osoba,Ruch,wysokosc jelca,pozycja sztychu,ustawienie stóp,lokcie blisko siebie,ruch noga do poz. koncowej,wyprostoawnie w lokciach,wyrzut rak do przodu,uderzenie odpowiednim ostrzem,Ocena 1,Ocena 2,Ocena 3
0,0,0,1,1,1,1,1,1,1,1,1,1,1
1,0,0,1,1,1,1,1,1,1,1,1,1,1
2,0,0,1,1,1,1,1,1,1,1,1,1,1
3,0,0,1,1,1,1,1,1,1,1,1,1,1
4,0,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781,14,4,1,0,1,0,1,1,1,1,1,0,1
3782,14,4,1,0,1,0,1,1,1,1,1,0,1
3783,14,4,1,0,1,0,1,1,1,1,1,0,1
3784,14,4,1,0,1,0,1,1,1,1,1,0,1
